## Introduction

I am a person living in Hong Kong. I am living in the Kennedy Town region, where it is very close to the underground station. The region has a balance of the eastern and western culture. It is easily accessible to the central downtown, but far enough to enjoy a quiet and relaxed environment. Recently, I have been invited by my boss to work in Toronto. The package is a nice deal and I decided to accept it. I am very excited, and at the same time very busy at the preparation work. I am looking for an apartment in Toronto which has a similar ambience compared to my current living environment. The question is, which Neighborhood should I look for?


## Business Problem

To find a neighborhood in Toronto that exhibits the closest characteristics compared to my current home: Kennedy town. The steps could include:

1. Getting the characteristics of Kennedy Town
2. Matching the characteristics of Kennedy Town to a neighborhood (or a few neighborhoods) in Toronto for consideration.

## Data

We will use the Toronto Data we have prepared in week 3, and get the characteristics of Kennedy Town from FourSquare.

### Import Libraries

In [6]:
import pandas as pd
import pickle
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

ModuleNotFoundError: No module named 'geopy'

### Toronto Data

In [5]:
with open(r'canada_postcodes_and_coordinates_df.pkl', 'rb') as f:
    toronto_df = pickle.load(f)

ModuleNotFoundError: No module named 'pandas.core.internals.managers'; 'pandas.core.internals' is not a package

In [ ]:
toronto_df = toronto_df.rename({'Neighbourhood': 'Neighborhood'}, axis=1)

In [ ]:
toronto_df.head

### Kennedy Town Data

#### Use geopy library to get the latitude and longitude values of Kennedy Town

In [ ]:
kennedy_town_address = 'Kennedy Town, Hong Kong'

geolocator = Nominatim(user_agent="explorer")
kt_location = geolocator.geocode(address)
kt_latitude = location.latitude
kt_longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(kt_latitude, kt_longitude))

#### Show a map of where Kennedy Town is

In [ ]:
map_kt = folium.Map(location=[kt_latitude, kt_longitude], zoom_start=10)

#### Get characteristics of Kennedy Town from FourSquare

In [ ]:
with open(r'foursquare_credentials.pkl', 'rb') as f:
    (CLIENT_ID, CLIENT_SECRET) = pickle.load(f)

In [ ]:
VERSION = '20180605' # Foursquare API version

Now, let's get the top 100 venues that are in Kennedy Town within a radius of 500 meters.

In [ ]:
radius = 500
LIMIT = 100


url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    kt_latitude, 
    kt_longitude, 
    radius, 
    LIMIT)

In [ ]:
results = requests.get(url).json()

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

And how many venues were returned by Foursquare?

In [ ]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))